In [1]:
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch import nn, optim
from torch.nn import functional as F
from dataset import get_mnist, separate
from skorch import NeuralNetClassifier
from sklearn.metrics import accuracy_score

os.environ['KERAS_BACKEND'] = 'torch'
import keras
from keras import layers

In [2]:
use_tqdm = True
tqdm = tqdm if use_tqdm else lambda x:x

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

tqdm, device

(tqdm.std.tqdm, 'cuda')

In [3]:
loader = get_mnist(batch_size=32)
len(loader[0]), len(loader[1])

(1875, 313)

In [4]:
x_train, y_train = separate(loader[0])
x_train.shape, y_train.shape

(torch.Size([60000, 1, 28, 28]), torch.Size([60000]))

In [5]:
x_test, y_test = separate(loader[1])
x_test.shape, y_test.shape

(torch.Size([10000, 1, 28, 28]), torch.Size([10000]))

In [6]:
class Model(nn.Module):
    
    def __init__(self, num_classes=10):
        super().__init__()
        self.model = keras.Sequential([layers.Input((1, 28, 28)),
                                       layers.Flatten(),
                                       layers.Dense(200, activation="relu"),
                                       layers.Dense(150, activation="relu"),
                                       layers.Dense(num_classes, activation="softmax")])
        
    def forward(self, x):
        return self.model(x)

In [7]:
model = Model()
pred_y = model(x_train[0:32])
pred_y.shape

torch.Size([32, 10])

In [8]:
# default loss: NLLL
net = NeuralNetClassifier(Model, max_epochs=100, lr=1e-3, 
                          batch_size=64, device=device)
net.fit(x_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.0919       0.5750        1.8786  3.3199
      2        1.6533       0.7163        1.4272  3.6940
      3        1.2355       0.7828        1.0625  3.5130
      4        0.9428       0.8214        0.8345  3.4099
      5        0.7660       0.8431        0.6972  3.6432
      6        0.6571       0.8562        0.6095  3.4766
      7        0.5855       0.8662        0.5494  3.5972
      8        0.5351       0.8712        0.5059  3.5042
      9        0.4978       0.8762        0.4729  3.6824
     10        0.4691       0.8812        0.4469  3.4660
     11        0.4462       0.8868        0.4260  3.3649
     12        0.4276       0.8898        0.4087  3.5164
     13        0.4121       0.8923        0.3942  3.4480
     14        0.3989       0.8951        0.3818  3.5472
     15        0.3876       0.8978        0.3711  3.6174
     16        0.3777       0.8

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Model(
    (model): <Sequential name=sequential_1, built=True>
  ),
)

In [9]:
y_pred = net.predict(x_test[:5])
y_pred

array([7, 2, 1, 0, 4], dtype=int64)

In [10]:
y_proba = net.predict_proba(x_test[:5])
y_proba

array([[2.87253089e-04, 6.51116494e-08, 1.57244783e-03, 4.62581497e-03,
        6.26708186e-07, 6.75435076e-05, 2.90309843e-09, 9.93242919e-01,
        2.36440646e-05, 1.79695126e-04],
       [1.35735760e-03, 5.47884338e-05, 9.90217090e-01, 4.85721650e-03,
        3.35569728e-09, 6.44412881e-04, 1.32739602e-03, 1.96229237e-08,
        1.54179730e-03, 1.53010440e-08],
       [3.84867781e-05, 9.82056081e-01, 5.07624680e-03, 2.78334250e-03,
        3.74092691e-04, 2.20517907e-03, 1.65344658e-03, 3.23489471e-03,
        2.16250122e-03, 4.15793387e-04],
       [9.99661446e-01, 6.19082785e-09, 1.19113945e-04, 1.27402373e-05,
        5.72676200e-08, 1.24894039e-04, 5.69772419e-05, 1.35747741e-05,
        3.50653022e-06, 7.71041323e-06],
       [2.77731975e-04, 1.30999451e-05, 1.42478058e-03, 1.21169527e-04,
        9.60752070e-01, 3.41278152e-04, 1.32051320e-03, 3.66160204e-03,
        3.69204528e-04, 3.17184590e-02]], dtype=float32)

In [11]:
y_pred = net.predict(x_test)
accuracy_score(y_test, y_pred)

0.9475

In [12]:
class Model(nn.Module):
    
    def __init__(self, num_classes=10):
        super().__init__()
        self.model = keras.Sequential([layers.Input((1, 28, 28)),
                                       layers.Flatten(),
                                       layers.Dense(200, activation="relu"),
                                       layers.Dense(150, activation="relu"),
                                       layers.Dense(num_classes)])
        
    def forward(self, x):
        return self.model(x)

In [13]:
net = NeuralNetClassifier(Model, max_epochs=100, lr=1e-3, 
                          criterion=nn.CrossEntropyLoss, 
                          batch_size=64, device=device)
net.fit(x_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.1237       0.5627        1.9309  3.2788
      2        1.7116       0.7337        1.4785  3.3370
      3        1.2727       0.8014        1.0796  3.4517
      4        0.9504       0.8328        0.8299  3.3002
      5        0.7599       0.8489        0.6855  3.2967
      6        0.6475       0.8609        0.5969  3.4231
      7        0.5759       0.8672        0.5380  3.5635
      8        0.5268       0.8721        0.4962  3.4567
      9        0.4910       0.8772        0.4649  3.4366
     10        0.4637       0.8822        0.4404  3.3310
     11        0.4420       0.8848        0.4207  3.3043
     12        0.4244       0.8875        0.4043  3.3430
     13        0.4096       0.8908        0.3906  3.4209
     14        0.3970       0.8935        0.3787  3.5442
     15        0.3861       0.8963        0.3683  3.6298
     16        0.3764       0.8

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Model(
    (model): <Sequential name=sequential_2, built=True>
  ),
)

In [14]:
y_pred = net.predict(x_test[:5])
y_pred

array([7, 2, 1, 0, 4], dtype=int64)

In [15]:
y_proba = net.predict_proba(x_test[:5])
y_proba

array([[9.5121060e-05, 4.4103300e-07, 1.3751260e-03, 5.6029479e-03,
        5.3078026e-07, 4.0260660e-05, 4.9278217e-09, 9.9266428e-01,
        9.0585327e-06, 2.1225419e-04],
       [9.0250460e-04, 3.3562185e-04, 9.7455263e-01, 8.6080274e-03,
        1.2145846e-07, 4.4421223e-03, 9.0637570e-03, 5.2779727e-09,
        2.0951836e-03, 3.6420548e-08],
       [2.3558026e-05, 9.8523468e-01, 5.1164851e-03, 1.6299746e-03,
        4.2730672e-04, 2.6301788e-03, 1.5003354e-03, 2.0845814e-03,
        1.1033247e-03, 2.4967152e-04],
       [9.9943763e-01, 2.2198115e-08, 3.8761937e-04, 1.7450084e-05,
        9.0123763e-08, 8.1105056e-05, 2.9317882e-05, 3.1880671e-05,
        2.2580439e-06, 1.2557837e-05],
       [5.3675211e-04, 1.1447048e-05, 4.0593916e-03, 1.7273609e-04,
        9.5664155e-01, 4.5337901e-04, 5.6341873e-03, 3.3983132e-03,
        1.7718109e-03, 2.7320391e-02]], dtype=float32)

In [16]:
y_pred = net.predict(x_test)
accuracy_score(y_test, y_pred)

0.9472